In [9]:
## This contains a pure IK based troting motion plan
## Author : Avadesh Meduri
## Date : 26/02/2021

import time
import numpy as np
import pinocchio as pin
import crocoddyl

from robot_properties_solo.config import Solo12Config
from py_biconvex_mpc.ik.inverse_kinematics import InverseKinematics
from py_biconvex_mpc.ik_utils.gait_generator import GaitGenerator


In [10]:
dt = 1e-2
T = 2.0

robot = Solo12Config.buildRobotWrapper()
q0 = np.array(Solo12Config.initial_configuration)
x0 = np.concatenate([q0, pin.utils.zero(robot.model.nv)])

pin.forwardKinematics(robot.model, robot.data, q0, pin.utils.zero(robot.model.nv))
pin.updateFramePlacements(robot.model, robot.data)

fl_loc = robot.data.oMf[robot.model.getFrameId("FL_FOOT")].translation
fr_loc = robot.data.oMf[robot.model.getFrameId("FR_FOOT")].translation
hl_loc = robot.data.oMf[robot.model.getFrameId("HL_FOOT")].translation
hr_loc = robot.data.oMf[robot.model.getFrameId("HR_FOOT")].translation


In [11]:
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=False)
viz.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [13]:
sl = np.array([0.0, 0.0, 0])
mass = pin.computeTotalMass(robot.model, robot.data)
com_loc = pin.centerOfMass(robot.model, robot.data, q0)
ctraj = np.zeros((int(T/dt), 6))
ctraj[:,0] = 3*mass*sl[0]/T

ori_traj = np.tile(np.eye(3), ((int(T/dt), 1, 1))) 

state_wt = np.array([0., 0, 10] + [1000] * 3 + [1.0] * (robot.model.nv - 6) \
                        + [0.00] * 3 + [100] * 3 + [0.5] *(robot.model.nv - 6))
q0 = np.array(Solo12Config.initial_configuration)
x_reg = np.concatenate([q0, pin.utils.zero(robot.model.nv)])


In [14]:
gg = GaitGenerator(robot,"NOne", T, dt)

# gg.create_centroidal_task(ctraj, 0, 2.0, "cmom_task", 1e5)

gg.create_swing_foot_task(fl_loc, fl_loc + sl, 0.0, 0.5, 0.1, "FL_FOOT", "FL_ftc1", 1e4)
gg.create_swing_foot_task(hr_loc, hr_loc + sl, 0.0, 0.5, 0.1, "HR_FOOT", "HR_ftc1", 1e4)
gg.create_contact_task(fr_loc, 0.0, 0.5, "FR_FOOT", "FR_ctc1", 1e3)
gg.create_contact_task(hl_loc, 0.0, 0.5, "HL_FOOT", "HL_ctc1", 1e3)

gg.create_swing_foot_task(fr_loc, fr_loc + 2*sl, 0.5, 1.0, 0.1, "FR_FOOT", "FR_ftc1", 1e4)
gg.create_swing_foot_task(hl_loc, hl_loc + 2*sl, 0.5, 1.0, 0.1, "HL_FOOT", "HL_ftc1", 1e4)
gg.create_contact_task(fl_loc + sl, 0.5, 1.0, "FL_FOOT", "FL_ctc1", 1e3)
gg.create_contact_task(hr_loc + sl, 0.5, 1.0, "HR_FOOT", "HR_ctc1", 1e3)

gg.create_swing_foot_task(fl_loc + sl, fl_loc + 3*sl, 1.0, 1.5, 0.1, "FL_FOOT", "FL_ftc1", 1e4)
gg.create_swing_foot_task(hr_loc + sl, hr_loc + 3*sl, 1.0, 1.5, 0.1, "HR_FOOT", "HR_ftc1", 1e4)
gg.create_contact_task(fr_loc + 2*sl, 1.0, 1.5, "FR_FOOT", "FR_ctc2", 1e3)
gg.create_contact_task(hl_loc + 2*sl, 1.0, 1.5, "HL_FOOT", "HL_ctc2", 1e3)

gg.create_swing_foot_task(fr_loc + 2*sl, fr_loc + 4*sl, 1.5, 2.0 - dt, 0.1, "FR_FOOT", "FR_ftc1", 1e4)
gg.create_swing_foot_task(hl_loc + 2*sl, hl_loc + 4*sl, 1.5, 2.0 - dt, 0.1, "HL_FOOT", "HL_ftc1", 1e4)
gg.create_contact_task(fl_loc + 3*sl, 1.5, 2.0, "FL_FOOT", "FL_vtc2", 1e3)
gg.create_contact_task(hr_loc + 3*sl, 1.5, 2.0, "HR_FOOT", "HR_vtc2", 1e3)

# gg.ik.add_orientation_tracking_task(robot.model.getFrameId("root_joint"), 0, 2.0, ori_traj, 1e5, "base_ori")
# comTrack = crocoddyl.CostModelCoMPosition(gg.ik.state, 4*sl + com_loc)
# gg.ik.terminalCostModel.addCost("comtask", comTrack, 1e+3)

xs, us = gg.optimize(x0, state_wt, x_reg, wt_xreg = 1e-2, wt_ureg = 1e-5)


full model
net time: 0.08683514595031738


In [14]:
for i in range(len(xs)):
    time.sleep(0.01)
    viz.display(xs[i][:robot.model.nq])

In [7]:
for i in range(len(xs)):
    print(xs[i][3:7])

[0. 0. 0. 1.]
[-8.92882262e-09 -4.29334661e-09 -3.34338728e-08  1.00000000e+00]
[-1.80006668e-08 -8.58006344e-09 -6.72393314e-08  1.00000000e+00]
[-2.72908760e-08 -1.28444267e-08 -1.01722612e-07  1.00000000e+00]
[-3.68410964e-08 -1.70687707e-08 -1.37227493e-07  1.00000000e+00]
[-4.66703532e-08 -2.12175670e-08 -1.74107105e-07  1.00000000e+00]
[-5.67856765e-08 -2.52323601e-08 -2.12730861e-07  1.00000000e+00]
[-6.71855193e-08 -2.90317914e-08 -2.53493509e-07  1.00000000e+00]
[-7.78613000e-08 -3.25142037e-08 -2.96826488e-07  1.00000000e+00]
[-8.88004234e-08 -3.55624916e-08 -3.43211642e-07  1.00000000e+00]
[-9.99926571e-08 -3.80525619e-08 -3.93196756e-07  1.00000000e+00]
[-1.11441367e-07 -3.98675778e-08 -4.47411330e-07  1.00000000e+00]
[-1.23181220e-07 -4.09204465e-08 -5.06579384e-07  1.00000000e+00]
[-1.35304001e-07 -4.11869062e-08 -5.71523937e-07  1.00000000e+00]
[-1.47993743e-07 -4.07509088e-08 -6.43155167e-07  1.00000000e+00]
[-1.61571450e-07 -3.98624362e-08 -7.22431646e-07  1.00000000e+